# SCOTUS justice tenure

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import timedelta
import altair as alt
from PyPDF2 import PdfFileWriter, PdfFileReader

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
url = "https://www.supremecourt.gov/about/members_text.aspx"

In [5]:
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
dfs = pd.read_html(page.text)

In [6]:
chiefs = pd.DataFrame(dfs[1])
associates = pd.DataFrame(dfs[2])

In [7]:
chiefs["type"] = "chief"
associates["type"] = "associate"

In [8]:
df = pd.concat([chiefs, associates])

In [9]:
df = df.drop(["Unnamed: 0"], axis=1).rename(
    columns={
        "Name": "name",
        "State App't From": "state",
        "Appointed by President": "president",
        "Judicial Oath Taken": "state_date",
        "Date Service Terminated": "end_date",
    }
)

In [10]:
df["state_date"] = pd.to_datetime(
    df["state_date"]
    .str.replace("(a)", "", regex=False)
    .str.replace("(b)", "", regex=False)
    .str.replace("(c)", "", regex=False)
)
df["end_date"] = df["end_date"].fillna("")

In [11]:
df["end_date"] = pd.to_datetime(
    df["end_date"]
    .str.replace("*", "", regex=False)
    .str.replace("(b)", "", regex=False)
    .str.replace("(c)", "", regex=False)
)

In [12]:
df[["last_name", "first_name", "suffix"]] = df["name"].str.split(", ", expand=True)

In [13]:
df["suffix"] = df["suffix"].fillna("")

In [14]:
df[
    [
        "first_name",
        "last_name",
        "suffix",
        "name",
        "state",
        "president",
        "state_date",
        "end_date",
        "type",
    ]
]

,first_name,last_name,suffix,name,state,president,state_date,end_date,type
0,John,Jay,,"Jay, John",New York,Washington,1789-10-19,1795-06-29,chief
1,John,Rutledge,,"Rutledge, John",South Carolina,Washington,1795-08-12,1795-12-15,chief
2,Oliver,Ellsworth,,"Ellsworth, Oliver",Connecticut,Washington,1796-03-08,1800-12-15,chief
3,John,Marshall,,"Marshall, John",Virginia,"Adams, John",1801-02-04,1835-07-06,chief
4,Roger Brooke,Taney,,"Taney, Roger Brooke",Maryland,Jackson,1836-03-28,1864-10-12,chief
5,Salmon Portland,Chase,,"Chase, Salmon Portland",Ohio,Lincoln,1864-12-15,1873-05-07,chief
6,Morrison Remick,Waite,,"Waite, Morrison Remick",Ohio,Grant,1874-03-04,1888-03-23,chief
7,Melville Weston,Fuller,,"Fuller, Melville Weston",Illinois,Cleveland,1888-10-08,1910-07-04,chief
8,Edward Douglass,White,,"White, Edward Douglass",Louisiana,Taft,1910-12-19,1921-05-19,chief
9,William Howard,Taft,,"Taft, William Howard",Connecticut,Harding,1921-07-11,1930-02-03,chief


In [15]:
today = pd.to_datetime("today").strftime(format="%Y-%m-%d")

In [16]:
df.loc[df["end_date"].isnull(), "end_date"] = today

In [17]:
df["years"] = ((df["end_date"] - df["state_date"]) / timedelta(days=365)).round(0)

In [18]:
df.sort_values("state_date")

,name,state,president,state_date,end_date,type,last_name,first_name,suffix,years
2,"Wilson, James",Pennsylvania,Washington,1789-10-05,1798-08-21,associate,Wilson,James,,9.0
0,"Jay, John",New York,Washington,1789-10-19,1795-06-29,chief,Jay,John,,6.0
3,"Blair, John",Virginia,Washington,1790-02-02,1795-10-25,associate,Blair,John,,6.0
1,"Cushing, William",Massachusetts,Washington,1790-02-02,1810-09-13,associate,Cushing,William,,21.0
0,"Rutledge, John",South Carolina,Washington,1790-02-15,1791-03-05,associate,Rutledge,John,,1.0
4,"Iredell, James",North Carolina,Washington,1790-05-12,1799-10-20,associate,Iredell,James,,9.0
5,"Johnson, Thomas",Maryland,Washington,1791-09-19,1793-01-16,associate,Johnson,Thomas,,1.0
6,"Paterson, William",New Jersey,Washington,1793-03-11,1806-09-09,associate,Paterson,William,,14.0
1,"Rutledge, John",South Carolina,Washington,1795-08-12,1795-12-15,chief,Rutledge,John,,0.0
7,"Chase, Samuel",Maryland,Washington,1796-02-04,1811-06-19,associate,Chase,Samuel,,15.0


In [19]:
presidents = {
    "Washington": "Unaffiliated",
    "Adams, John": "Federalist",
    "Jackson": "Democratic",
    "Lincoln": "Republican",
    "Grant": "Republican",
    "Cleveland": "Democratic",
    "Taft": "Republican",
    "Harding": "Republican",
    "Hoover": "Republican",
    "Roosevelt, F.": "Democratic",
    "Truman": "Democratic",
    "Eisenhower": "Republican",
    "Nixon": "Republican",
    "Reagan": "Republican",
    "Bush, G. W.": "Republican",
    "Jefferson": "Democratic-Republican",
    "Madison": "Democratic-Republican",
    "Monroe": "Democratic-Republican",
    "Adams, J. Q.": "Democratic-Republican",
    "Van Buren": "Democratic",
    "Tyler": "Whig",
    "Polk": "Democratic",
    "Fillmore": "Whig",
    "Pierce": "Democratic",
    "Buchanan": "Democratic",
    "Hayes": "Republican",
    "Garfield": "Republican",
    "Arthur": "Republican",
    "Harrison": "Republican",
    "McKinley": "Republican",
    "Roosevelt, T.": "Republican",
    "Wilson": "Democratic",
    "Coolidge": "Republican",
    "Kennedy": "Democratic",
    "Johnson, L.": "Democratic",
    "Ford": "Republican",
    "Bush, G. H. W.": "Republican",
    "Clinton": "Democratic",
    "Obama": "Democratic",
    "Trump": "Republican",
}

In [20]:
df["pres_party"] = df["president"].map(presidents)

In [21]:
df[df["type"] == "associate"].sort_values("state_date", ascending=False).head(25)

,name,state,president,state_date,end_date,type,last_name,first_name,suffix,years,pres_party
102,"Barrett, Amy Coney",Indiana,Trump,2020-10-27,2022-03-14,associate,Barrett,Amy Coney,,1.0,Republican
101,"Kavanaugh, Brett M.",Maryland,Trump,2018-10-06,2022-03-14,associate,Kavanaugh,Brett M.,,3.0,Republican
100,"Gorsuch, Neil M.",Colorado,Trump,2017-04-10,2022-03-14,associate,Gorsuch,Neil M.,,5.0,Republican
99,"Kagan, Elena",Massachusetts,Obama,2010-08-07,2022-03-14,associate,Kagan,Elena,,12.0,Democratic
98,"Sotomayor, Sonia",New York,Obama,2009-08-08,2022-03-14,associate,Sotomayor,Sonia,,13.0,Democratic
97,"Alito, Samuel A., Jr.",New Jersey,"Bush, G. W.",2006-01-31,2022-03-14,associate,Alito,Samuel A.,Jr.,16.0,Republican
96,"Breyer, Stephen G.",Massachusetts,Clinton,1994-08-03,2022-03-14,associate,Breyer,Stephen G.,,28.0,Democratic
95,"Ginsburg, Ruth Bader",New York,Clinton,1993-08-10,2020-09-18,associate,Ginsburg,Ruth Bader,,27.0,Democratic
94,"Thomas, Clarence",Georgia,"Bush, G. H. W.",1991-10-23,2022-03-14,associate,Thomas,Clarence,,30.0,Republican
93,"Souter, David H.",New Hampshire,"Bush, G. H. W.",1990-10-09,2009-06-29,associate,Souter,David H.,,19.0,Republican


In [22]:
alt.Chart(df[(df["state_date"] > "1900-01-01")]).mark_bar().encode(
    x="years:Q",
    y=alt.Y("name", sort="-x"),
    color=alt.Color(
        "pres_party",
        scale=alt.Scale(
            domain=["Democratic", "Republican"], range=["#1851ac", "#c62222"]
        ),
    ),
)

alt.Chart(...)

In [23]:
df[df["state_date"] > "1970-01-01"]["pres_party"].value_counts()

Republican    15
Democratic     4
Name: pres_party, dtype: int64